In [1]:
from __future__ import division, print_function, absolute_import
import tensorflow as tf

#Importing tensorflow MNIST dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
#Training Parameters
learning_rate = 0.001
num_steps = 200 #How many total steps
batch_size = 128 #How many batchs to send in each epoch
display_step = 10 #How many steps to display training loss and accuracy data

In [3]:
#Network Parameters
num_input = 784 #28x28 - data inpu size
num_classes = 10 #0-9 10 digits
#dropout = 0.75 #Dropout, probability to keep units

In [4]:
#tf Graph input variables
#NONE - which is the first dimension, corresponding to the batch size, indicates it  can be any size
X = tf.placeholder(tf.float32, [None, num_input]) #The inputs = the number of pixels in the image
Y = tf.placeholder(tf.float32, [None, num_classes]) #The output = the number of classifications 0-9 digits
#keep_prob = tf.placeholder(tf.float) #Dropout (keep probability)

In [9]:
def conv2d(x, W, b, strides=1):
    #Conv2D Wrapper - allows for simplificty
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    '''
    Padding 'SAME' -> the output tensor should have the same w and h values as input, replacing any lost w or h with 
    0's to preserve the original w and h
    '''
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    #MaxPooling Wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding = 'SAME')

In [6]:
#Creating the actual model
def conv_net(x, weights, biases):
    # MNIST data is 1-D Vector of 784 features 28*28
    # Reshape to match picture format [Height X Width X Channel]
    # Tensor input become 4-D: [batch size, height, width, channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    #Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    #Max Pooling
    conv1 = maxpool2d(conv1, k=2)
    
    #Convolution Layers
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    #Max Pooling
    conv2 = maxpool2d(conv2, k=2)
    
    #Fully Connected Layer
    #Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    #Apply Dropout
    #fc1 = tf.nn.dropout(fc1, dropout)
    
    #Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    
    return out

In [7]:
weights = {
    #5X5 Conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),
    #5X5 Conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # Fully Connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [10]:
#Constructing the model
logits = conv_net(X, weights, biases)
prediction = tf.nn.softmax(logits)

In [12]:
#Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [13]:
#Evaluate Model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [14]:
#Initialize the variables
init = tf.global_variables_initializer()

In [15]:
#Start Training
with tf.Session() as sess:
    
    #Run the initializer
    sess.run(init)
    
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        #Run optimization operation (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            #Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss = " + "{:.4f}".format(loss) + ", Training Accuracy = " + \
                 "{:.3f}".format(acc))
    
    print("Optimization Finished")
    
    #Calculate Accuracy in 256 MNIST test images
    print("Testing Accuracy: ", \
         sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256]}))

Step 1, Minibatch Loss = 65072.5469, Training Accuracy = 0.102
Step 10, Minibatch Loss = 19200.9180, Training Accuracy = 0.312
Step 20, Minibatch Loss = 10384.6016, Training Accuracy = 0.516
Step 30, Minibatch Loss = 5301.3560, Training Accuracy = 0.656
Step 40, Minibatch Loss = 4472.8853, Training Accuracy = 0.742
Step 50, Minibatch Loss = 5123.1143, Training Accuracy = 0.781
Step 60, Minibatch Loss = 3757.6907, Training Accuracy = 0.766
Step 70, Minibatch Loss = 2646.5286, Training Accuracy = 0.828
Step 80, Minibatch Loss = 1930.6824, Training Accuracy = 0.859
Step 90, Minibatch Loss = 2560.7671, Training Accuracy = 0.836
Step 100, Minibatch Loss = 2495.1118, Training Accuracy = 0.812
Step 110, Minibatch Loss = 2385.8940, Training Accuracy = 0.859
Step 120, Minibatch Loss = 1639.8282, Training Accuracy = 0.875
Step 130, Minibatch Loss = 1550.7310, Training Accuracy = 0.859
Step 140, Minibatch Loss = 1985.3110, Training Accuracy = 0.883
Step 150, Minibatch Loss = 1752.7467, Training A